### Imports

In [16]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, circuit_drawer
import numpy as np
import matplotlib.pyplot as plt
import heapq
from IPython.display import clear_output
import time

## Define The Maze

### Maze Representation
The maze is represented as a 2D list in Python, where each sub-list represents a row in the maze. The elements of the sub-lists represent the different types of cells in the maze. 

#### Elements of the Maze
1. **Start Point ('S'):** The starting position in the maze is marked with the character `'S'`.
   - This is where the search begins.

2. **Goal Point ('G'):** The goal or endpoint of the maze is marked with the character `'G'`.
   - The objective is to navigate from `'S'` to `'G'`.

3. **Walls (1):** Cells with the value `1` represent walls or obstacles.
   - These cells cannot be traversed.

4. **Open Path (0):** Cells with the value `0` represent open paths.
   - These cells can be traversed to navigate through the maze.

#### Navigation Rules
   - The algorithm must find a path from `'S'` to `'G'`.
   - Movement is restricted to cells with a value of `0`.
   - Walls (`1`) block movement.

#### Possible Movements
   - **Up**
   - **Down**
   - **Left**
   - **Right**



#### Maze

In [17]:
maze = [
    ['S', 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 'G']
]


#### Define the start and goal positions


In [18]:
start = (0, 0)  # Top-left corner
goal = (14, 14)   # Bottom-right corner


# Classical Search (A*)

## Heuristic Function for A* Algorithm

The `heuristic` function is a key component of the A* pathfinding algorithm. It estimates the cost of the cheapest path from the current node to the goal.

### Explanation

1. **Purpose**:
   - The heuristic function calculates the Manhattan distance between two points `a` and `b` on a grid.
   - It is used to guide the A* algorithm by providing an estimate of the remaining cost to reach the goal.

2. **Parameters**:
   - `a`: A tuple `(x1, y1)` representing the coordinates of the current node.
   - `b`: A tuple `(x2, y2)` representing the coordinates of the goal node.

3. **Calculation**:
   - The Manhattan distance is calculated as:
     
     distance = |x_1 - x_2| + |y_1 - y_2|
     
   - This assumes movement is allowed only in the four cardinal directions (up, down, left, right).

4. **Return Value**:
   - The function returns the computed Manhattan distance as an integer.

### Why Use Manhattan Distance?
- The Manhattan distance is suitable for grids where diagonal movement is not allowed.
- It provides a simple and efficient way to estimate the distance between two points, ensuring the A* algorithm remains optimal and complete.


In [19]:
def heuristic(a, b):
    """Heuristic function for A* (Manhattan distance)."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

##### Get Neighbors Function

The `get_neighbors` function is used in pathfinding algorithms to determine valid neighboring cells for a given position in the maze.

### Explanation

1. **Purpose**:
   - The function identifies all valid neighboring cells that can be traversed from the current position.
   - It helps in exploring possible moves in the maze during pathfinding.

2. **Parameters**:
   - `maze`: A 2D list representing the maze where `0` indicates an open path and `1` indicates a wall or barrier.
   - `position`: A tuple `(x, y)` representing the current position in the maze.

3. **Logic**:
   - The function iterates over four possible movements: **Up**, **Down**, **Left**, **Right**.
   - For each direction, it calculates the potential new position `(nx, ny)`.
   - It checks if the new position is within bounds and not blocked by a wall (`maze[nx][ny] != 1`).

4. **Return Value**:
   - Returns a list of valid neighboring positions as tuples `(nx, ny)`. These positions can be visited next.


In [20]:
def get_neighbors(maze, position):
    """Get valid neighbors for the current position."""
    rows, cols = len(maze), len(maze[0])
    x, y = position
    neighbors = []

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # Up, Down, Left, Right
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] != 1:  # Check bounds and barriers
            neighbors.append((nx, ny))

    return neighbors


#### Path Construction

In [21]:
def reconstruct_path(came_from, current):
    """Reconstruct the path from start to goal."""
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path


#### A* Algorithm function

#### Parameters:
- `maze` (list of list of ints): A 2D grid where each cell is either 0 (open space) or 1 (wall).
- `start` (tuple): Coordinates (x, y) of the starting position in the maze.
- `goal` (tuple): Coordinates (x, y) of the goal position in the maze.

#### Returns:
- A list of coordinates representing the path from the start to the goal if a path is found.
- None if no path is found.

In [22]:
def astar(maze, start, goal):
    """A* algorithm with debug information."""
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        _, current = heapq.heappop(open_set)

        # Debugging: print current exploration status
        print(f"Exploring: {current}, f_score: {f_score[current]}")

        # Check if the goal is reached
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(maze, current):
            tentative_g_score = g_score[current] + 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

        # Debugging: print open set status after exploring neighbors
        print(f"Open set: {open_set}")

    return None  # No path found


#### Helper functions

In [23]:
def show_path(maze, path):
    """
    Visualize the maze with the path.
    - Path cells are marked with '*'.
    - Start is marked with 'S'.
    - Goal is marked with 'G'.
    """
    maze_copy = [row[:] for row in maze]  # Create a copy of the maze to modify

    for x, y in path:
        if maze_copy[x][y] != 'G':  # Avoid overwriting the goal marker
            maze_copy[x][y] = '*'

    start_x, start_y = path[0]
    maze_copy[start_x][start_y] = 'S'  # Mark the start position

    for row in maze_copy:
        print(" ".join(str(cell) for cell in row))


In [24]:
def print_maze(maze):
    """
    Prints the maze in a human-readable format.
    - Open spaces are shown as '0'.
    - Barriers are shown as '1'.
    - Start is shown as 'S'.
    - Goal is shown as 'G'.
    """
    for row in maze:
        print(" ".join(str(cell) for cell in row))
    print("\n")  # Add a newline for better readability


##### Visualization

In [25]:
def print_exploration(maze, open_set):
    """
    Visualize the maze during A* exploration.
    - Explored cells in the open_set are marked as 'E'.
    """
    maze_copy = [row[:] for row in maze]
    for _, (x, y) in open_set:
        if maze_copy[x][y] not in ('S', 'G'):  # Avoid overwriting start/goal markers
            maze_copy[x][y] = 'E'

    print_maze(maze_copy)
    time.sleep(0.5)  # Pause for visualization


In [26]:
def clear_terminal():
    """Clear the output in a Jupyter Notebook."""
    clear_output(wait=True)


#### A* With vizualisation

In [27]:
def astar_with_visualization(maze, start, goal):
    """
    A* algorithm with visualization of exploration 
    """
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        # Clear the output to update the maze state
        clear_terminal()
        print("Exploring the maze...")

        # Visualize the current state
        print_exploration(maze, open_set)

        _, current = heapq.heappop(open_set)

        # Check if the goal is reached
        if current == goal:
            return reconstruct_path(came_from, current)

        for neighbor in get_neighbors(maze, current):
            tentative_g_score = g_score[current] + 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None  # No path found

#### Execute

In [28]:
# Run the A* algorithm with visualization
path = astar_with_visualization(maze, start, goal)

# Show the final path if found
if path:
    print("Path found:")
    show_path(maze, path)
else:
    print("No path found.")


Exploring the maze...
S 0 0 1 0 0 1 1 1 0 0 1 1 1 0
1 1 0 1 0 1 0 0 0 0 1 0 0 0 0
0 E 0 1 E 1 1 1 1 0 0 1 1 0 1
1 1 0 0 0 0 0 0 1 1 E 0 0 0 0
0 1 1 1 1 1 1 0 0 0 0 1 1 1 E
0 0 0 0 0 1 0 1 1 1 0 0 0 0 0
0 1 1 1 0 1 0 0 0 E 0 1 1 1 0
0 0 0 1 0 0 0 1 1 1 0 1 0 E 0
1 1 0 1 1 1 0 1 0 E 0 1 0 1 0
0 0 0 0 0 1 0 1 0 1 0 1 E E 0
1 1 1 1 0 0 0 0 0 1 0 0 0 1 0
0 0 0 1 1 1 1 1 0 1 1 1 0 1 0
0 1 1 1 0 0 0 0 0 0 0 1 0 1 0
0 0 0 0 0 1 1 1 0 1 0 E 0 1 0
1 1 1 1 0 0 0 0 0 1 0 E 0 0 G


Path found:
S S * 1 0 0 1 1 1 0 0 1 1 1 0
1 1 * 1 0 1 0 0 0 0 1 0 0 0 0
0 0 * 1 0 1 1 1 1 0 0 1 1 0 1
1 1 * * * * * * 1 1 0 0 0 0 0
0 1 1 1 1 1 1 * * * * 1 1 1 0
0 0 0 0 0 1 0 1 1 1 * * * * *
0 1 1 1 0 1 0 0 0 0 0 1 1 1 *
0 0 0 1 0 0 0 1 1 1 0 1 0 0 *
1 1 0 1 1 1 0 1 0 0 0 1 0 1 *
0 0 0 0 0 1 0 1 0 1 0 1 0 0 *
1 1 1 1 0 0 0 0 0 1 0 0 0 1 *
0 0 0 1 1 1 1 1 0 1 1 1 0 1 *
0 1 1 1 0 0 0 0 0 0 0 1 0 1 *
0 0 0 0 0 1 1 1 0 1 0 0 0 1 *
1 1 1 1 0 0 0 0 0 1 0 0 0 0 G


# Quantum Search